# Notes

Code for producing GOIT pipelines summary stats, and for calculating landing page stats

In [1]:
import pandas
pandas.set_option("display.max_rows", 50, "display.max_columns", 50)

import numpy
import pygsheets
import datetime
import re

In [2]:
pandas.io.formats.excel.ExcelFormatter.header_style = None

#fuel_type = 'Gas'
fuel_type = 'Oil'
#fuel_type = 'NGL'

if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-Gas-'+str(datetime.date.today())+'.xlsx', engine='xlsxwriter')
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-NGL-'+str(datetime.date.today())+'.xlsx', engine='xlsxwriter')
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-Oil-'+str(datetime.date.today())+'.xlsx', engine='xlsxwriter')

## import data

In [3]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
spreadsheet = gc.open_by_key('1IQ_g9PCr6pexDvEBoi5jboNfamJtPQ63ezxyq0qRsu0') # for June 2022

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig[pipes_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig[owners_df_orig['ProjectID']!='']
owners_df_orig = owners_df_orig[owners_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig[owners_df_orig.Status!='N/A']

owners_df_orig.set_index('ProjectID', inplace=True)

In [4]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)


pipes_df_orig.replace('--',numpy.nan,inplace=True)

In [5]:
region_df_orig = spreadsheet.worksheet('title', 'Region dictionary').get_as_df()

In [6]:
region_df_eu = region_df_orig.loc[region_df_orig['EuropeanUnion']=='Yes']
region_df_egt = region_df_orig.loc[region_df_orig['EuroGasTracker']=='Yes']
region_df_europe = region_df_orig.loc[region_df_orig['Region']=='Europe']
region_df_eu_uk = region_df_orig.loc[(region_df_orig['Region']=='Europe') | 
                                          (region_df_orig['Country'].isin(['United Kingdom','Israel']))]
#region_df_global = region_df_orig.copy()

In [7]:
region_df_touse = region_df_orig.copy()

### create country-specific dataframes for region, country_ratios_df, owners_df

In [8]:
country_ratios_df_touse = country_ratios_df.copy()#.loc[country_ratios_df['Country'].str.contains(
                                           # '|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.copy()#.loc[owners_df_orig['Countries'].str.contains(
                                        #    '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.copy()#loc[pipes_df_orig['Countries'].str.contains(
                                    #        '|'.join(region_df_touse['Country'].tolist()))]

### sum MergedKmByCountry and MergedKmByRegion

In [9]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [10]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [11]:
country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
totals_row.name = 'Total'
km_by_region = km_by_region.append(totals_row)

totals_row = km_by_country.sum(axis=0)
totals_row.name = 'Total'
km_by_country = km_by_country.append(totals_row)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [12]:
km_by_region

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Region,,,,,,,,,
Australia and New Zealand,0.00,0.00,0.00,0.00,0.00,1878.30,0.00,0.00,187.00
East Asia,1046.59,1486.08,2532.67,0.00,8477.06,26292.25,0.00,0.00,6825.55
Eurasia,2094.20,768.23,2862.43,0.00,2635.32,46616.13,0.00,52.00,618.00
Europe,799.90,207.00,1006.90,0.00,8232.21,27955.13,141.55,0.00,0.00
Latin America and the Caribbean,0.00,0.00,0.00,262.00,3730.84,13281.02,0.00,425.00,0.00
Middle East and North Africa,1816.74,1903.00,3719.74,0.00,3587.42,26810.56,0.00,2368.08,1214.51
North America,2913.14,1380.88,4294.02,8090.76,30867.84,126096.14,0.00,0.00,2698.42
SE Asia,240.49,715.50,955.99,306.00,0.00,2159.86,0.00,0.00,0.00
South Asia,1194.00,2316.00,3510.00,249.40,1692.10,9987.18,0.00,0.00,0.00


## pipeline km by parent company (owner) and project status

In [13]:
owner_parent_calculations_df = pandas.DataFrame()
# needs country, km in each country columns as well

for idx,row in country_ratios_df_subset.iterrows():
    #print(row.ComboID)
    parent_string = row.Parent
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'Country':row.Country,
                                                                         'Status':row.Status,
                                                                         'MergedKmByCountry':row.MergedKmByCountry}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.MergedKmByCountry

### pipeline km by start year, type

In [14]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Gas')]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Oil')]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='NGL')]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [15]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2022)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2022)), columns=['Oil pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2022)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

totals_row = km_by_start_year.sum(axis=0)
totals_row.name = 'Total'
km_by_start_year = km_by_start_year.append(totals_row)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')

## save excel file

In [16]:
excel_writer.save()

## calculating stats for landing page

In [17]:
# number of projects tracked in total
print(pipes_df_orig.loc[pipes_df_orig['Fuel']=='Oil'].shape[0], 'oil pipeline projects tracked')
print(pipes_df_orig.loc[pipes_df_orig['Fuel']=='Oil']['LengthMergedKm'].sum(), 'km tracked')

910 oil pipeline projects tracked
402400.29 km tracked


In [18]:
# number of projects tracked in total
print(pipes_df_orig.loc[pipes_df_orig['Fuel']=='NGL'].shape[0], 'NGL pipeline projects tracked')
print(pipes_df_orig.loc[pipes_df_orig['Fuel']=='NGL']['LengthMergedKm'].sum(), 'km tracked')

61 NGL pipeline projects tracked
29052.520000000008 km tracked


In [19]:
pipes_df_oil = pipes_df_orig.loc[pipes_df_orig['Fuel']=='Oil']
pipes_df_oil.replace('',numpy.nan,inplace=True)

pipes_df_ngl = pipes_df_orig.loc[pipes_df_orig['Fuel']=='NGL']
pipes_df_ngl.replace('',numpy.nan,inplace=True)

/Users/baird/miniconda3/envs/gem/lib/python3.9/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
